In [57]:
from pathlib import Path
import json

import numpy as np
import pandas as pd
import geopandas as gpd
from townsnet.provision.model import UrbanFunctionCalculator

In [66]:
CITY_INFO_PATH = Path("data/city/towns.geojson")
SERVICE_FILES = Path('data/services')
OUTPUT_JSON = Path("data/city/city_profiles.json")

In [83]:
model = UrbanFunctionCalculator()
model.load_city_info(CITY_INFO_PATH)
model.city_info.index = list(range(207, len(model.city_info) + 207))

model.load_service_results(SERVICE_FILES)
model.compute_ml_features(n_clusters=4, components=2, with_anomaly=True)
profiles = model.build_profiles(min_export_threshold=0)

print(f"Сформировано профилей: {len(profiles)}")

Сформировано профилей: 2931


In [84]:
from pprint import pprint
pprint(profiles[1822])

{'ML': {'anomaly': 0.05473444013054862,
        'cluster': 2,
        'embedding': [7.515064459096589, -5.509628303608313],
        'index': 89.54377823213943},
 'geometry': <POINT (30.657 59.973)>,
 'Градообразующие функции': {'Безопасность': {'Обслуженное население': 0.0},
                             'Здравоохранение': {'Обслуженное население': 17.0},
                             'Культура и отдых': {'Обслуженное население': 0.0},
                             'Образование': {'Обслуженное население': 782.0},
                             'Социальная помощь': {'Обслуженное население': 0.0},
                             'Спорт': {'Обслуженное население': 0.0},
                             'Туризм': {'Обслуженное население': 35.0},
                             'Услуги': {'Обслуженное население': 307.0}},
 'Градообслуживающие функции': {'Безопасность': {'Обеспеченность, %': 100.0,
                                                 'Обслуженное население': 1},
                               

In [85]:
gdf = model.get_profilies()
gdf[gdf['Потенциальный опорный пункт'] == True]

,city_id,Название,Население,Опорный пункт,Потенциальный опорный пункт,Лучшая градообразующая функция,"Лучшая градообразующая функция, чел",geometry
208,208,Большой Остров,116,False,True,Туризм,96.0,POINT (33.786 59.47518)
236,236,Сельхозтехника,116,False,True,Услуги,1791.0,POINT (33.8095 59.48441)
243,243,Головачово,59,False,True,Здравоохранение,109.0,POINT (35.43393 59.51748)
246,246,Заборье,61,False,True,Услуги,1519.0,POINT (35.24354 59.52142)
263,263,Ольеши,58,False,True,Услуги,1918.0,POINT (35.4089 59.6967)
...,...,...,...,...,...,...,...,...
3113,3113,Староселье,94,False,True,Туризм,496.0,POINT (31.17724 59.61569)
3118,3118,Ям-Ижора,6778,False,True,Образование,162.0,POINT (30.57305 59.70478)
3119,3119,Войскорово,6776,False,True,Образование,171.0,POINT (30.56024 59.69105)
3121,3121,Тельмана,6663,False,True,Услуги,7910.0,POINT (30.61175 59.72616)


In [ ]:
mf = model.ml_features.copy()
X = model._build_city_feature_matrix()

# Пороговая логика
idx_thr = np.quantile(mf["index"], 0.75)
anom_thr = np.quantile(mf["anomaly_score"].dropna(), 0.95) if "anomaly_score" in mf else np.inf

labels = []
for cid, row in mf.iterrows():
    idx_h = row["index"] >= idx_thr
    anom_h = ("anomaly_score" in row) and (row.get("anomaly_score", np.nan) >= anom_thr)
    if idx_h and not anom_h:
        lab = "high_index_low_anomaly"
    elif idx_h and anom_h:
        lab = "high_index_high_anomaly"
    elif (not idx_h) and anom_h:
        lab = "low_index_high_anomaly"
    else:
        lab = "low_index_low_anomaly"
    labels.append((cid, lab))

groups = pd.DataFrame(labels, columns=["city_id","quadrant"]).set_index("city_id")

# Диагностика отличий: средние по сервисам для каждой группы
def summarize_group(name):
    members = groups.index[groups["quadrant"] == name]
    if len(members) == 0:
        return None, None
    Xg = X.loc[members]
    mean_diff = (Xg.mean() - X.mean()).sort_values(ascending=False)
    top_up = mean_diff.head(5)
    top_down = mean_diff.tail(5)
    return top_up, top_down

for q in ["high_index_low_anomaly","high_index_high_anomaly","low_index_high_anomaly","low_index_low_anomaly"]:
    up, down = summarize_group(q)
    print(f"\n=== {q} ===")
    if up is None:
        print("Нет городов в группе")
        continue
    print("Топ + отличий (среднее - общее среднее):")
    display(up)
    print("Топ - отличий:")
    display(down)
    

In [ ]:
# OUTPUT_JSON.parent.mkdir(parents=True, exist_ok=True)
# model.save_city_json(OUTPUT_JSON, by="id")
# print(f"Результат сохранён в {OUTPUT_JSON}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

for group, data in model.ml_features.groupby('cluster'):
    plt.hist(
        data['anomaly_score'],
        bins=30,
        alpha=0.5,
        label=f'Cluster {group}',
    )

plt.xlabel('Anomaly Score')
plt.ylabel('Count')
plt.legend()
plt.tight_layout()
plt.show()


plt.figure(figsize=(10, 6))
for group, data in model.ml_features.groupby('cluster'):
    plt.hist(
        data['index'],
        bins=30,
        alpha=0.5,
        label=f'Cluster {group}',
    )

plt.xlabel('index')
plt.ylabel('Count')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
for group, data in model.ml_features.groupby('cluster'):
    plt.scatter(
        data['index'],
        data['anomaly_score'],
        alpha=0.6,
        label=f'Cluster {group}',
        s=20
    )

plt.axhline(0, color='black', linewidth=1)
plt.xlabel('Index')
plt.ylabel('Anomaly Score')
plt.legend()
plt.tight_layout()
plt.show()


plt.figure(figsize=(10, 6))
plt.scatter(
        model.ml_features['pca1'],
        model.ml_features['pca2'],
        alpha=0.6,
        label=f'Cluster {group}',
        s=20
    )
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.tight_layout()
plt.show()

In [ ]:
# Анализ: приоритетное развитие и потенциально опорные пункты (ПОП)
from IPython.display import display

ml = model.ml_features.copy()
if 'anomaly_score' not in ml.columns:
    ml['anomaly_score'] = np.nan

idx_hi = ml['index'].quantile(0.75)
idx_lo = ml['index'].quantile(0.25)
anom_hi = ml['anomaly_score'].quantile(0.90) if ml['anomaly_score'].notna().any() else np.inf
anom_lo = ml['anomaly_score'].quantile(0.30) if ml['anomaly_score'].notna().any() else -np.inf

ml['quadrant'] = (
    np.where(ml['index'] >= idx_hi, 'high', 'low') + '_index_' +
    np.where(ml['anomaly_score'] >= anom_hi, 'high', 'low') + '_anomaly'
)

print(f"anomaly high: {anom_hi:.2f}, low: {anom_lo:2f}")

In [ ]:
# Агрегированные показатели по всем сервисным группам
totals = []
for df_group in model.group_aggregates.values():
    totals.append(df_group[['demand','served','external_demand']])
totals = pd.concat(totals).groupby(level=0).sum()
totals['overall_provision_pct'] = (totals['served'] / totals['demand']).replace([np.inf,-np.inf], np.nan).clip(0, 1) * 100
totals['external_share_pct'] = (totals['external_demand'] / totals['demand']).replace([np.inf,-np.inf], np.nan).clip(lower=0) * 100

In [ ]:
base = pd.concat([
    ml[['index','anomaly_score','quadrant']],
    totals[['overall_provision_pct','external_share_pct']],
    model.city_info[['city_name','population','is_anchor', 'geometry']],
], axis=1)
base['external_supply'] = model._external_supply_total.reindex(base.index).fillna(0.0)
base['external_supply_per_1k'] = base['external_supply'] / (base['population'].replace(0, np.nan) / 1000)
base['external_supply_per_1k'] = base['external_supply_per_1k'].replace([np.inf,-np.inf], np.nan)

In [ ]:
# Города, требующие первостепенного развития
ext_thr = base['external_share_pct'].quantile(0.75)
prov_thr = base['overall_provision_pct'].quantile(0.25)
need_mask = (
    (base['index'] <= idx_lo) |
    (base['overall_provision_pct'] <= prov_thr) |
    (base['external_share_pct'] >= ext_thr)
)
if ml['anomaly_score'].notna().any():
    need_mask |= base['anomaly_score'] >= anom_hi
base['priority_score'] = (
    (100 - base['index']).clip(lower=0) * 0.4 +
    base['external_share_pct'].fillna(0) * 0.3 +
    (100 - base['overall_provision_pct']).clip(lower=0) * 0.3
)
priority_cities = base[need_mask].sort_values('priority_score', ascending=False)
print('Города, требующие первостепенного развития:')
display(priority_cities[['city_name','index','overall_provision_pct','external_share_pct','anomaly_score','priority_score','quadrant']])

In [ ]:
# Потенциально опорные пункты
supply_thr = base['external_supply_per_1k'].quantile(0.75)
pop_mask = (
    (base['external_supply_per_1k'] >= supply_thr) &
    (base['index'] >= idx_hi) &
    (~base['is_anchor'])
)
if ml['anomaly_score'].notna().any():
    pop_mask &= base['anomaly_score'] <= anom_lo
pop_candidates = base[pop_mask].sort_values(['external_supply_per_1k','index'], ascending=[False, False])
pop_candidates = gpd.GeoDataFrame(pop_candidates, geometry='geometry', crs=model.city_info.crs)
print('Потенциально опорные пункты :')
display(pop_candidates[['city_name','index','external_supply','external_supply_per_1k','anomaly_score','quadrant']])


In [ ]:
pop_candidates.explore(tiles='CartoDB Positron')